# Data Management with Kosh

After generating data with Maestro or Merlin we need to access the data in our Kosh/Sina store. After some data manipulation we can train a surrogate model to emulate the Pyranda physics calculations.

The Kosh store gives us the convenience of saving all the variables and outputs with the associated metadata. Instead of saving varibales like the Atwood number and velocity-magnitude in the filenames we can save all the information we need in the metadata. Then it's convenient to find in our ensemble later. 

In [ ]:
import numpy as np
import matplotlib
import kosh
import os
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
import matplotlib.pyplot as plt
from matplotlib import cm


# Connect to the Kosh store
store_dir = os.path.join(os.getcwd(), "experiments/pyranda.sql")
store = kosh.connect(store_dir)

# Create an ensemble or use an existing one
# We can associate all our datasets and images with this ensemble
name = os.listdir(os.path.join(os.getcwd(), "RT_STUDIES"))[0]
try:
    ensemble = next(store.find_ensembles(name=name))
except Exception:
    # Does not exist yet
    ensemble = store.create_ensemble(name=name)

print(f"ensemble: {ensemble}")

## Choosing times to evaluate mixing width

We gathered a lot of data with the simulation runs. However, we need to choose specific points in time to evaluate mixing width. Here we can choose the number of time points. 

In [ ]:
# Choose number of times here
NTpts = 1
# All simulations run from 0.0 to at least 60.0 seconds
Tmin = 0.0
Tmax = 60.0

# The time points are evenly spaced between 0.0 and 60.0
sample_times = np.linspace(Tmin, Tmax, NTpts)
# If we only want one point we will evaluate at the max time
if len(sample_times) == 1:
    sample_times = [Tmax]
    
print(f"Model evaluation time/s {sample_times}")

## Gathering variables from each simulation run

We can loop through the datasets in our Kosh store, and save the inputs and outputs of interest. Data of all different types can be associated together in our ensemble.

In [ ]:
samples = []
N_cases = len(list(store.find(types="pyranda", run_type="sim", ids_only=True)))
for i, case in enumerate(store.find(types="pyranda", run_type="sim"), start=1):
    # Let's add this dataset to our ensemble
    #print("*********************************")
    #print("DS:", case.id)
    #print("*********************************")
    ensemble.add(case)

    # Let's retrieve the variables of interest
    time = case["variables/time"][:] # Time
    width = case["variables/mixing width"][:] # Width
    mixed = case["variables/mixedness"][:] # Mixedness
    atwood = case.atwood_number
    velocity = case.velocity_magnitude
    lbl = f"Vel: {velocity} - At: {atwood}"
    plt.figure(2)
    plt.plot(time, width, "-o", label=lbl)
    for st in sample_times:
        plt.axvline(x=st, color='b', label=f"{st} s")
    plt.xlabel("Time")
    plt.ylabel("Mix Width")
    plt.title("Rayleigh-Taylor Simulations")
    if i == N_cases:
        fnm = "all_mixing_width.png"
        ensemble.associate(fnm, "png", metadata={"title": lbl})

    # Plotting to show the input sampling design
    plt.figure(1)
    plt.plot(atwood, velocity, 'ko')
    plt.xlabel("Atwood number")
    plt.ylabel("Velocity magnitude")
    plt.title("Latin Hypercube Space-Filling Design")
    if i == N_cases:
        fnm = "atwood_vs_vel.png"
        ensemble.associate(fnm, "png", metadata={"title":'atwood vs velocity'})
            
    # For each time, qoi, get NTpts
    #  Sample = [atwood, velocity, w(0), w(1), w(2) ...]
    sample_widths = np.interp(sample_times, time, width)
    sample = np.insert( sample_widths, 0, atwood)
    sample = np.insert( sample, 1, velocity)
    samples.append( sample )

plt.show()
        
samples = np.array(samples)

# Save for next step
header = f"# 'atwood' 'velocity' "
for ii in range(NTpts):
    header += " 'width-%s' " % ii
fnm = f"rt_sim_data.csv"
np.savetxt(fnm, samples, delimiter=',',header=header)
#associate with ensemble
ensemble.associate(fnm, "pandas/csv", metadata={"gp_data":True})

print(f"Data size: {samples.shape}")
print("First 5 rows")
print(samples[:5])

## Fitting the Gaussian Process (GP) Models

We will fit a Gaussian process surrogate model for each time we chose from the simulation. This model will need to be able to predict mixing width very quickly and accurately. The Gaussian process model can return a prediction and a standard error estimate. The error should be very small for data points it was trained on, and larger when it has to interpolate between training data points. 

In [ ]:
# We need an array of model inputs for the GP
xgp = samples[:,0:2]
# The GP model performs better when the inputs are scaled
scaler = MMS()
scaled_samples = scaler.fit_transform(xgp)

# Get inputs for 2D plots
# We're going to evaluate the model at points it was not trained on
atwoods    = np.linspace(.25,.75, 100)
velocities = np.linspace(.75, 1.25, 100)
at2d, vel2d = np.meshgrid(atwoods, velocities)
atwoods = at2d.flatten().reshape(-1,1)
velocities = vel2d.flatten().reshape(-1,1)
inputs = np.concatenate( (atwoods, velocities), axis=1 )
scaled_inputs = scaler.transform(inputs)

## Plotting the GP Predictions

Here we're going to evaluate the model at many inputs to see how well it can predict mixing width. The mix width prediction is shown as a blue response surface over the Atwood and velocity values. The color of the surface indicates low error blue to high error red. The error is very low in the areas where the model was trained, but we forced to model to predict outside of the range it was trained on and you can see the error increases at the edges where it did not have traning data.

In [ ]:
GP_times = []

# Fitting a GP model for NTpts in time
for ii in range(NTpts):
    sample_time = sample_times[ii]
    y = samples[:, 2 + ii]  # Get width at this time-slice
    GP_times.append(GPR().fit(scaled_samples, y))

    # See GP prediction in 2D
    pred, std = GP_times[ii].predict(scaled_inputs, return_std=True)

    fig_num = 3 + ii
    fig = plt.figure(fig_num)
    ax = fig.add_subplot(111, projection='3d')
    pred2d = pred.reshape(at2d.shape)
    std2d = std.reshape(at2d.shape)
    mycol = cm.jet( 1.5*(std2d-std.min())/(std.max()-std.min())    )
    ax.plot_surface(at2d, vel2d, pred2d,facecolors=mycol)
    ax.set_xlabel('Atwood')
    ax.set_ylabel('Velocity')
    ax.set_zlabel('Width')
    fnm = f"GP_at_{sample_time}_s.png"
    ensemble.associate(fnm, "png", metadata={"title":'2D GP'})

## Leave-One-Out Cross Validation

We can evaluate the GP model with a leave-one-out method where a model is trained on all the data except the first data point, next we train a model on all but the second data point, etc. Each time we train a model we evaluate it on the data point that was left out. It's ok to use in this situation because our model is fast to train and predict. 

In [ ]:
loo = LeaveOneOut()

outputs = samples[:, 2:]

time_point_error = []
for ii in range(NTpts):
    loo_pred = []
    loo_bar = []
    loo_sqerror = []
    for i, (train_index, test_index) in enumerate(loo.split(scaled_samples)):
        y = outputs[:, ii]
        gp_model = GPR().fit(scaled_samples[train_index, :], y[train_index])
        pred, std = gp_model.predict(scaled_samples[test_index, :], return_std=True)
        loo_pred.append(pred)
        loo_bar.append((pred + std * 1.96) - (pred - std * 1.96))
        loo_sqerror.append((y[test_index] - pred)**2)
    plt.figure(3 + NTpts + ii)
    plt.errorbar(outputs[:, ii].flatten(), np.array(loo_pred).flatten(), yerr=np.array(loo_bar).flatten(), fmt='o',label='GP')
    plt.plot([2,7],[2,7],'r-',label="Exact")
    plt.xlabel("Actual Mix Width")
    plt.ylabel("Predicted Mix Width")
    plt.title(f"GP Model at {sample_times[ii]} s")
    plt.legend()
    print(f"MSE at {sample_times[ii]} s: {sum(loo_sqerror)/len(loo_sqerror)}")